# Лабораторная работа №3 «Алгоритм Шора»


### Вариант 2

Выполнил Проскуряков Роман Владимирович

In [1]:
!pip install qiskit qiskit-aer --quiet

In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi
import matplotlib.pyplot as plt

import numpy as np
import math
from qiskit import transpile


In [3]:
def simulate(qc, shots=1024):
    simulator = AerSimulator()
    qc = transpile(qc, simulator, optimization_level=0)
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    print("Результаты измерений:", counts)
    return counts

def draw_and_simulate(qc, title):
    print(f"{title}")
    print(qc.draw(output='text'))  # текстовая визуализация схемы
    return simulate(qc)

def go(qc, title="", shots=1024, noise_model=None):
    print(f"{title}")
    print(qc.draw(output='text'))  # текстовая визуализация схемы
    simulator = AerSimulator(noise_model=noise_model)
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    print("Результаты измерений:", counts)
    return counts

In [4]:
# Параметры
N = 20
a = 7

# Проверяем, что a и N взаимно просты
if math.gcd(a, N) != 1:
    print(f"НОД({a}, {N}) = {math.gcd(a, N)} - уже делитель!")
else:
    print(f"a = {a} взаимно просто с N = {N}")

n_count = N.bit_length()  # количество кубитов для поиска периода
n_work = n_count   # количество кубитов для представления чисел до 12
phase_qbits_list = list(range(n_count))
work_qbits_list = list(range(n_count, n_count + n_work))

# Создаём квантовую схему
qc = QuantumCircuit(n_count + n_work, n_count)

a = 7 взаимно просто с N = 20


In [5]:
# 1. Инициализация: рабочий регистр в |1>
qc.x(n_count + n_work - 1)

In [6]:
# 2. Применяем Адамары к фазовым кубитам
for q in range(n_count):
    qc.h(q)

## Реализуем генерацию неконтролируемого оператора U (через матрицу)

In [7]:
def create_U_operator(N, a, power=1):
  """
  Создает оператор U^power для функции f(x) = a^x mod N
  U^power|y⟩ = |(a^power × y) mod N⟩
  """
  # Количество кубитов для представления чисел
  n_work = N.bit_length()
  # Размер матрицы: 2^n_work x 2^n_work
  dim = 2**n_work

  # Вычисляем a^power mod N
  a_power = pow(a, power, N)

  # Создаем матрицу
  U_matrix = np.zeros((dim, dim), dtype=complex)

  for y in range(dim):
    if y < N:
      target = (a_power * y) % N
    else:
      target = y
    U_matrix[target][y] = 1

  # Проверяем, что матрица унитарна
  identity = np.eye(dim)
  is_unitary = np.allclose(U_matrix @ U_matrix.conj().T, identity)
  if not is_unitary:
    print(f"Матрица неунитарна: N={N}, a={a}, power={power}")

  gate = qi.Operator(U_matrix).to_instruction()
  gate.label = f"U^{power}"

  return gate

def create_cU_operator(a, power=1):
  """
  Создает контролируемую версию оператора U^power
  """
  c_U = create_U_operator(N, a, power).control(1)
  return c_U


# Тестируем оператор сU

In [8]:
# Тестируем оператор cU^power
power = 1
print(f"=== Тестирование оператора cU^{power} для N={N}, a={a} ===")
cU_gate = create_cU_operator(a, power)

# Создаем схему для проверки
qc_test = QuantumCircuit(1 + n_count, 1 + n_count)

# Подготовим состояние |4⟩ (двоичное 0100)
qc_test.x(0)  # включаем
qc_test.x(3)  # |0100⟩ = |4⟩

test_cU_list = list(range(0, 1 + n_count))

# Применяем оператор U
qc_test.append(cU_gate, test_cU_list)

# Измеряем
qc_test.measure(test_cU_list, test_cU_list)

# Запускаем симуляцию
counts = draw_and_simulate(qc_test, f"Проверка работы оператора cU^{power}")

# Интерпретируем результат
for bitstring in counts:
  st = bitstring[:-1]
  decimal = int(st, 2)  # переворачиваем биты для правильного порядка
  print(f"   Двоичное: {st} -> Десятичное: {decimal}")
  print(f"   Ожидали: 2 (двоичное 00010)")

=== Тестирование оператора cU^1 для N=20, a=7 ===
Проверка работы оператора cU^1
     ┌───┐        ┌─┐               
q_0: ┤ X ├───■────┤M├───────────────
     └───┘┌──┴───┐└╥┘┌─┐            
q_1: ─────┤0     ├─╫─┤M├────────────
          │      │ ║ └╥┘┌─┐         
q_2: ─────┤1     ├─╫──╫─┤M├─────────
     ┌───┐│      │ ║  ║ └╥┘┌─┐      
q_3: ┤ X ├┤2 U^1 ├─╫──╫──╫─┤M├──────
     └───┘│      │ ║  ║  ║ └╥┘┌─┐   
q_4: ─────┤3     ├─╫──╫──╫──╫─┤M├───
          │      │ ║  ║  ║  ║ └╥┘┌─┐
q_5: ─────┤4     ├─╫──╫──╫──╫──╫─┤M├
          └──────┘ ║  ║  ║  ║  ║ └╥┘
c: 6/══════════════╩══╩══╩══╩══╩══╩═
                   0  1  2  3  4  5 
Результаты измерений: {'010001': 1024}
   Двоичное: 01000 -> Десятичное: 8
   Ожидали: 2 (двоичное 00010)


In [9]:
# Тестируем оператор U^1
print(f"=== Тестирование оператора U для N={N}, a={a} ===")
U_gate = create_U_operator(N, a)

# Создаем схему для проверки
qc_test = QuantumCircuit(n_count, n_count)

# Подготовим состояние |4⟩ (двоичное 0100)
qc_test.x(2)  # |0100⟩ = |4⟩

# Применяем оператор U
qc_test.append(U_gate, phase_qbits_list)

# Измеряем
qc_test.measure(phase_qbits_list, phase_qbits_list)

# Запускаем симуляцию
counts = draw_and_simulate(qc_test, "Проверка работы оператора U")

# Интерпретируем результат
for bitstring in counts:
  decimal = int(bitstring, 2)  # переворачиваем биты для правильного порядка
  print(f"  Двоичное: {bitstring} -> Десятичное: {decimal}")
  print(f"  Ожидали: 8 (двоичное 01000)")

=== Тестирование оператора U для N=20, a=7 ===
Проверка работы оператора U
          ┌──────┐┌─┐            
q_0: ─────┤0     ├┤M├────────────
          │      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 U^1 ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
          │      │ ║  ║  ║ └╥┘┌─┐
q_4: ─────┤4     ├─╫──╫──╫──╫─┤M├
          └──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 
Результаты измерений: {'01000': 1024}
  Двоичное: 01000 -> Десятичное: 8
  Ожидали: 8 (двоичное 01000)


In [10]:
# Тестируем оператор U^power
power = 2
print(f"=== Тестирование оператора U^{power} для N={N}, a={a} ===")
U_gate = create_U_operator(N, a, power)

# Создаем схему для проверки
qc_test = QuantumCircuit(n_count, n_count)

# Подготовим состояние |4⟩ (двоичное 0100)
qc_test.x(2)  # |0100⟩ = |4⟩

# Применяем оператор U
qc_test.append(U_gate, phase_qbits_list)

# Измеряем
qc_test.measure(phase_qbits_list, phase_qbits_list)

# Запускаем симуляцию
counts = draw_and_simulate(qc_test, "Проверка работы оператора U^2")

# Интерпретируем результат
for bitstring in counts:
  decimal = int(bitstring, 2)  # переворачиваем биты для правильного порядка
  print(f"   Двоичное: {bitstring} -> Десятичное: {decimal}")
  print(f"   Ожидали: 16 (двоичное 10000)")

=== Тестирование оператора U^2 для N=20, a=7 ===
Проверка работы оператора U^2
          ┌──────┐┌─┐            
q_0: ─────┤0     ├┤M├────────────
          │      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 U^2 ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
          │      │ ║  ║  ║ └╥┘┌─┐
q_4: ─────┤4     ├─╫──╫──╫──╫─┤M├
          └──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 
Результаты измерений: {'10000': 1024}
   Двоичное: 10000 -> Десятичное: 16
   Ожидали: 16 (двоичное 10000)


## Применяем управляемый оператор U

In [11]:
for j in range(n_count):
  power = 2**j  # 2^0, 2^1, 2^2, ...

  # Создаем управляемый оператор U^(2^j)
  cU_gate = create_cU_operator(a, power)

  # Добавляем в схему
  # Контрольный кубит - j-й фазовый кубит
  # Целевые кубиты - все рабочие кубиты
  qc.append(cU_gate, [j] + work_qbits_list)

  print(f"  Добавлен оператор U^{power}, контролируемый кубитом {j}")

print(qc.draw(output='text'))

  Добавлен оператор U^1, контролируемый кубитом 0
  Добавлен оператор U^2, контролируемый кубитом 1
  Добавлен оператор U^4, контролируемый кубитом 2
  Добавлен оператор U^8, контролируемый кубитом 3
  Добавлен оператор U^16, контролируемый кубитом 4
     ┌───┐                                         
q_0: ┤ H ├───■─────────────────────────────────────
     ├───┤   │                                     
q_1: ┤ H ├───┼───────■─────────────────────────────
     ├───┤   │       │                             
q_2: ┤ H ├───┼───────┼───────■─────────────────────
     ├───┤   │       │       │                     
q_3: ┤ H ├───┼───────┼───────┼───────■─────────────
     ├───┤   │       │       │       │             
q_4: ┤ H ├───┼───────┼───────┼───────┼────────■────
     └───┘┌──┴───┐┌──┴───┐┌──┴───┐┌──┴───┐┌───┴───┐
q_5: ─────┤0     ├┤0     ├┤0     ├┤0     ├┤0      ├
          │      ││      ││      ││      ││       │
q_6: ─────┤1     ├┤1     ├┤1     ├┤1     ├┤1      ├
          │      ││  

## Применение обратного квантового преобразования Фурье (QFT^(-1))

In [12]:
# Вычислить обратное квантовое преобразование Фурье
def qft_dagger(qc, n):
  for qubit in range(n//2):
    qc.swap(qubit, n-qubit-1)
  for j in range(n):
    for m in range(j):
      qc.cp(-np.pi/float(2**(j-m)), m, j)
    qc.h(j)

  qc.name = "QFT†"

qft_dagger(qc, n_count)

In [13]:
qc.measure(phase_qbits_list, phase_qbits_list)

In [14]:
counts = simulate(qc, 2000)

Результаты измерений: {'00000': 453, '10000': 492, '11000': 535, '01000': 520}


## Метод непрерывных дробей

In [15]:
# 1. Находим самую частую битовую строку
most_frequent = max(counts, key=counts.get)
print(f"Самая частая битовая строка: {most_frequent}")

Самая частая битовая строка: 11000


In [16]:
# 2. Переводим в число и дробь
x = int(most_frequent, 2)
phi = x / (2**n_count)
print(f"Число: {x}, φ = {phi}")

Число: 24, φ = 0.75


In [17]:
# # 3. Пробуем разные r от 1 до N-1
# for r_candidate in range(1, N):
#     # Находим ближайшее s для этого r
#     s = round(phi * r_candidate)

#     # Проверяем, насколько точно дробь s/r_candidate приближает φ
#     if abs(phi - s/r_candidate) < 0.01:  # допуск 1%
#         # Проверяем, является ли r_candidate периодом
#         if pow(a, r_candidate, N) == 1:
#             print(f"Найден период r = {r_candidate}")
#             r = r_candidate
#             break

# print(f"\nИскомый период r = {r}")

In [18]:
from fractions import Fraction
def getR():
  frac = Fraction(phi).limit_denominator(N)

  r = frac.denominator  # знаменатель — это наш кандидат в r
  s = frac.numerator    # числитель — это s

  # Шаг 4: Проверяем r
  if r % 2 == 0 and pow(a, r, N) == 1:
      return r
  else:
      # Если не подходит, пробуем r/2, r/4, ... пока не найдём чётный период
      while r > 1:
          if r % 2 == 0:
              r_test = r // 2
              if pow(a, r_test, N) == 1:
                  return r_test
          r //= 2

      return None  # не нашли подходящий период

r = getR()
print(f"\nИскомый период r = {r}")


Искомый период r = 4


In [19]:
x = a**(r//2) % N

div1 = math.gcd(x - 1, N)
div2 = N // div1
div3 = math.gcd(x + 1, N)
div4 = N // div3
all_div = {div1, div2, div3, div4}
print("Найденные делители числа N =", all_div)


Найденные делители числа N = {10, 2, 4, 5}
